In [1]:
%%capture
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install litellm

In [2]:
import os, sys
from openai import OpenAI

if not os.environ["OPENAI_API_KEY"]: 
    os.environ["OPENAI_API_KEY"]= '<REDACTED>'

MODEL = "gpt-4o-mini"

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

Baseline decision tree:

In [3]:
class Node:
    def __init__(self, question=None, tag = None, yes_branch=None, no_branch=None, examples=None, summary=None):
        self.question = question
        self.yes_branch = yes_branch
        self.no_branch = no_branch
        self.examples = examples
        self.summary = summary
        self.tag = tag

def print_tree(node, indent=0):
    """ Recursively print the decision tree. """
    if node is None:
        return
    if node.summary:
        # Print leaf node summary
        print(' ' * indent + f'→ Summary: The keyword is {node.summary}.')
    if node.examples:
        # Print leaf node examples
        print(' ' * indent + f'→ Examples: {", ".join(node.examples)}')
    if node.question:
        # Print the question node
        turn = indent // 4 + 1
        print(' ' * indent + f'Turn {turn}: {node.question}')
        if node.yes_branch or node.no_branch:
            print(' ' * indent + ' ├─ Yes:')
            print_tree(node.yes_branch, indent + 4)  # Print yes branch
            print(' ' * indent + ' └─ No:')
            print_tree(node.no_branch, indent + 4)   # Print no branch

# Define the decision tree structure
tree = Node(
    "Is it a place?",
    tag = "place",
    yes_branch=Node(
        examples=['Lesotho', 'Acapulco', 'Oceania', 'Parking lot'],
        question="Is it a country?",
        tag = "country",
        yes_branch=Node(
            examples=['Brazil', 'Japan', 'Kenya', 'Norway'],
            question="Is this country either in Asia or in Africa?",
            yes_branch=Node(
                examples=['Thailand', 'North Korea', 'Egypt', 'Nigeria'],
                question="Is this country in Asia?",
                yes_branch=Node(summary='a country in Asia', examples=['Kazakhstan', 'South Korea']),
                no_branch=Node(summary='a country in Africa', examples=['Morocco', 'Ghana'])
            ),
            no_branch=Node(
                examples=['Lithuania', 'Papua New Guinea', 'Uruguay', 'Slovakia'],
                question="Is this country in Europe?",
                yes_branch=Node(summary='a country in Europe', examples=['Romania', 'Germany']),
                no_branch=Node(
                    examples=['Australia', 'Nicaragua', 'Chile', 'United States'],
                    question="Is this country in the Americas?",
                    yes_branch=Node(summary='a country in the Americas', examples=['Argentina', 'Canada']),
                    no_branch=Node(summary='a country in Oceania', examples=['New Zealand', 'Samoa'])
                )
            )
        ),
        no_branch=Node(
            examples=['Venice', 'Lagos', 'Times Square', 'Colosseum'],
            question="Is it a city?",
            tag = "city",
            yes_branch=Node(
                examples=['Paris', 'Astana', 'Alexandria', 'Sao Paulo'],
                question="Is it a capital city?",
                yes_branch=Node(summary='a capital city', examples=['Moscow', 'Santiago']),
                no_branch=Node(summary='a city which is not a capital', examples=['Chicago', 'Tianjin'])
            ),
            no_branch=Node(
                examples=['Oceania', 'Nevada', 'Times Square', 'Colosseum'],
                question="Is it either a continent or a state of a country?",
                yes_branch=Node(
                    examples=['North America', 'Nevada', 'Roraima', 'Uttar Pradesh'],
                    question="Is it a continent?",
                    yes_branch=Node(summary='a continent', examples=['Africa', 'Asia']),
                    no_branch=Node(summary='a state of a country', examples=['Kentucky', 'Santa Catarina Brazil'])
                ),
                no_branch=Node(
                    examples=['Copacabana beach', 'university campus', 'Antarctica', 'tundra'],
                    question="Is it a location made by humans?",
                    yes_branch=Node(summary='a location made by humans', examples=['Champs-Elysees', 'Amusement park']),
                    no_branch=Node(summary='a natural location', examples=['Amazon rainforest', 'Desert'])
                )
            )
        )
    ),
    no_branch=Node(
        examples=['Pencil sharpener', 'cloud', 'chimney', 'electric vehicle'],
        question="Is it man-made?",
        tag="man-made",
        yes_branch=Node(
            examples=['plastic bottle', 'martini', 'earbuds', 'helicopter'],
            question="Is it found indoors?",
            tag="indoors",
            yes_branch=Node(
                examples=['pencil sharpener', 'refrigerator', 'coffee machine', 'escalator'],
                question="Is it something found in a home?",
                tag="home",
                yes_branch=Node(summary='a man-made thing found in people\'s homes', examples=["Bed frame", "Stove"]),
                no_branch=Node(summary='a man-made thing found indoors but not in people\'s homes', examples=["Hospital bed", "Erlenmeyer flask"])
            ),
            no_branch=Node(
                examples=['pickup truck', 'train', 'fountain', 'obelisk'],
                question="Is it something found in a city?",
                tag="city",
                yes_branch=Node(summary='a man-made thing found outdoors in cities', examples=['Apartment Building', 'Fire Hydrant']),
                no_branch=Node(summary='a man-made thing found outdoors outside of cities', examples=['Satellite', 'Barn'])
            ),
        ),
        no_branch=Node(
            examples=['thunder', 'Acacia', 'giraffe', 'river'],
            question="Is it a living thing?",
            tag="living",
            yes_branch=Node(
                examples=['Palm tree', 'beetle', 'whale', 'apple'],
                question="Is it an animal?",
                tag="animal",
                yes_branch=Node(
                    examples=['sea lion', 'cape toad', 'kangaroo', 'peacock'],
                    question="Is it a mammal or a bird?",
                    yes_branch=Node(summary='either a mammal or a bird', examples=["Elephant", "Seagull"]),
                    no_branch=Node(summary='either a fish, insect, reptilian or amphibian', examples=["Shark", "Wasp", "Frog", "Alligator"])
                ),
                no_branch=Node(
                    examples=['mold', 'deciduous tree', 'yeast', 'agave'],
                    question="Is it a plant?",
                    tag="plant",
                    yes_branch=Node(
                        examples=['orchid', 'poison ivy', 'maple tree', 'sunflower'],
                        question="Is it a tree?",
                        yes_branch=Node(summary='a tree', examples=["Oak tree", "Cactus"]),
                        no_branch=Node(summary='a plant which is not a tree', examples=["Rose", "Fern"])
                    ),
                    no_branch=Node(
                        examples=['black mold', 'truffle', 'cremini mushroom', 'bacillus'],
                        question="Is it a fungus?",
                        yes_branch=Node(summary='a fungus', examples=["Wood ear mushroom", "Yeast"]),
                        no_branch=Node(summary='a bacteria', examples=['E Coli', 'Streptococcus'])
                    )
                )
            ),
            no_branch=Node(
                examples=['uranus', 'seine river', 'carbon', 'electron'],
                question="Is it a geological feature?",
                tag="geological feature",
                yes_branch=Node(
                    examples=['Mount Fuji', 'volcano', 'canyon', 'Rocky mountains'],
                    question="Is it a body of water?",
                    yes_branch=Node(summary='a body of water', examples=["River", "Mediterranean Sea"]),
                    no_branch=Node(summary='a geological feature which is not a body of water', examples=["Cave", "Volcano"])
                ),
                no_branch=Node(
                    examples=['mercury', 'sun', 'iron', 'lead'],
                    question="Is it a chemical element, molecule or compound?",
                    yes_branch=Node(summary='a chemical element, molecule or compound', examples=["Salt", "Oxygen"]),
                    no_branch=Node(summary='a non-living thing not made by humans', examples=['Pebble', 'Milky Way'])
                )
            )
        )
    )
)

# Print the decision tree
print_tree(tree)

Turn 1: Is it a place?
 ├─ Yes:
    → Examples: Lesotho, Acapulco, Oceania, Parking lot
    Turn 2: Is it a country?
     ├─ Yes:
        → Examples: Brazil, Japan, Kenya, Norway
        Turn 3: Is this country either in Asia or in Africa?
         ├─ Yes:
            → Examples: Thailand, North Korea, Egypt, Nigeria
            Turn 4: Is this country in Asia?
             ├─ Yes:
                → Summary: The keyword is a country in Asia.
                → Examples: Kazakhstan, South Korea
             └─ No:
                → Summary: The keyword is a country in Africa.
                → Examples: Morocco, Ghana
         └─ No:
            → Examples: Lithuania, Papua New Guinea, Uruguay, Slovakia
            Turn 4: Is this country in Europe?
             ├─ Yes:
                → Summary: The keyword is a country in Europe.
                → Examples: Romania, Germany
             └─ No:
                → Examples: Australia, Nicaragua, Chile, United States
                Turn 5

Open keywords data:

In [5]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()
    return lines

keywords_list = read_file_to_list('20qs-data/keywords.txt')
keywords_list = keywords_list[1:]
print(f"Number of keywords in data: {len(keywords_list)}")

Number of keywords in data: 2046


## Label Data

In [15]:
from collections import Counter
from tqdm import tqdm
import time

def get_answer_from_node(node, keyword):
    
    prompt = f"Keyword: {keyword}\n\nQuestion: {node.question}\n\n"
    prompt += "\nPlease provide an answer to the question based on the keyword. ONLY ANSWER Yes OR No. IF UNSURE, CHOOSE MOST LIKELY ANSWER FROM Yes OR No."
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=10
    )
    response = response.choices[0].message.content

    return response


def label_keywords(node, keywords, labels, results):
    
    question_tag = node.tag
    yes_no_counts = Counter()

    for keyword in tqdm(keywords, desc = 'label_keywords', position = 0, leave = True, ncols = 100):
        answer = get_answer_from_node(node, keyword)
        answer = answer.lower()
        # Count yes/no answers
        if 'yes' in answer:
            yes_no_counts['yes'] += 1
            answer = 'yes'
        elif 'no' in answer:
            yes_no_counts['no'] += 1
            answer = 'no'
        else:
            yes_no_counts['err'] +=1
            answer = 'err'
        if keyword in labels:
            labels[keyword][question_tag] = answer
        else:
            labels[keyword] = {question_tag: answer}

    # Calculate percentages
    total_answers = sum(yes_no_counts.values())
    yes_percentage = (yes_no_counts['yes'] / total_answers) * 100 if total_answers > 0 else 0
    no_percentage = (yes_no_counts['no'] / total_answers) * 100 if total_answers > 0 else 0
    err_percentage = (yes_no_counts['err'] / total_answers) * 100 if total_answers > 0 else 0

    # Print results
    print(node.question)
    print(f"Yes answers: {yes_no_counts['yes']}, percentage: {yes_percentage:.2f}%")
    print(f"No answers: {yes_no_counts['no']}, percentage: {no_percentage:.2f}%")
    print(f"Error answers: {yes_no_counts['err']}, percentage: {err_percentage:.2f}%")

    # Save results
    results[question_tag] = {
        'yes_percentage': yes_percentage,
        'no_percentage': no_percentage,
        'value_counts': dict(yes_no_counts)
    }

    return labels, results

Read current keyword labels and results:

In [8]:
import json

def read_jsonl_and_transform(file_path):
    result_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            record = json.loads(line)
            keyword = record.pop('key')
            result_dict[keyword] = record

    return result_dict

def read_json_to_dict(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

keyword_labels, results = read_jsonl_and_transform('data/labeled_keywords.jsonl'), read_json_to_dict('data/labeling_results.jsonl')

Label places/things:

In [72]:
keyword_labels, results = label_keywords(tree, keywords_list, keyword_labels, results)

Is it a place?
Yes answers: 719, percentage: 35.15892420537897%
No answers: 1326, percentage: 64.84107579462103%
Error answers: 0, percentage: 0.0%


In [12]:
keyword_places = [keyword for keyword in keywords_list if keyword_labels[keyword]['place'] == 'yes']
keyword_things = [keyword for keyword in keywords_list if keyword_labels[keyword]['place'] == 'no']
things_branch = tree.no_branch
places_branch = tree.yes_branch

print(f"Number of keywords labeled as things: {len(keyword_things)}.")

1327

Label man-made living:

In [100]:
keyword_labels, results = label_keywords(not_man_made_branch, keyword_not_man_made, keyword_labels, results)

Is it a living thing?
Yes answers: 182, percentage: 64.31095406360424%
No answers: 101, percentage: 35.68904593639576%
Error answers: 0, percentage: 0.0%


In [104]:
keyword_living = [keyword for keyword in keyword_not_man_made if keyword_labels[keyword]['living'] == 'yes']
keyword_not_living = [keyword for keyword in keyword_not_man_made if keyword_labels[keyword]['living'] == 'no']
living_branch = not_man_made_branch.yes_branch
not_living_branch = not_man_made_branch.no_branch

Label indoors:

In [110]:
keyword_labels, results = label_keywords(man_made_branch, keyword_man_made, keyword_labels, results)

Is it found indoors?
Yes answers: 912, percentage: 87.44%
No answers: 131, percentage: 12.56%
Error answers: 0, percentage: 0.00%


In [111]:
keyword_indoors = [keyword for keyword in keyword_man_made if keyword_labels[keyword]['indoors'] == 'yes']
keyword_outdoors = [keyword for keyword in keyword_man_made if keyword_labels[keyword]['indoors'] == 'no']

Categorical labels:

In [9]:
category_nodes = [
    Node(question='Is it broadly related to food, drinks or cooking?', tag='food, drinks, cooking'),
    Node(question='Is it broadly related to arts, sports or entertainment?', tag='arts, sports, entertainment'),
    Node(question='Is it broadly related to clothing, beauty or accessories?', tag='clothing, beauty, accessories'),
    Node(question='Is it broadly related to furniture or architecture?', tag='furniture, architecture'),
    Node(question='Is it broadly related to cleaning or hygiene?', tag='cleaning, hygiene'),
    Node(question='Is it broadly related to transportation or vehicles?', tag='transportation, vehicles'),
    Node(question='Is it broadly related to electronics or technology?', tag='electronics, technology'),    
    Node(question='Is it broadly related to agriculture or industry?', tag='agriculture, industry'),
    Node(question='Is it broadly related to science, school or education?', tag='science, school, education'),
    Node(question='Is it broadly related to health or safety?', tag='health, safety'),
]

In [149]:
for cat_node in tqdm(category_nodes, desc = 'categories', ncols=150):
    keyword_labels, results = label_keywords(cat_node, keyword_things, keyword_labels, results)

categories:  10%|█████████▉                                                                                         | 1/10 [12:19<1:50:51, 739.07s/it]

Is it broadly related to food, drinks or cooking?
Yes answers: 392, percentage: 29.56%
No answers: 934, percentage: 70.44%
Error answers: 0, percentage: 0.00%


categories:  20%|███████████████████▊                                                                               | 2/10 [24:16<1:36:52, 726.57s/it]

Is it broadly related to arts, sports or entertainment?
Yes answers: 439, percentage: 33.11%
No answers: 887, percentage: 66.89%
Error answers: 0, percentage: 0.00%


categories:  30%|█████████████████████████████▋                                                                     | 3/10 [36:28<1:25:01, 728.73s/it]

...

Is it broadly related to furniture or architecture?
Yes answers: 172, percentage: 12.97%
No answers: 1154, percentage: 87.03%
Error answers: 0, percentage: 0.00%


categories:  50%|████████████████████████████████████████████████                                                | 5/10 [2:18:21<3:20:56, 2411.26s/it]

Is it broadly related to cleaning or hygiene?
Yes answers: 295, percentage: 22.25%
No answers: 1031, percentage: 77.75%
Error answers: 0, percentage: 0.00%


categories:  50%|████████████████████████████████████████████████                                                | 5/10 [2:18:36<2:18:36, 1663.33s/it]


KeyboardInterrupt: 

TODO: 
- Finish adding the labels below.

In [17]:
for cat_node in tqdm(category_nodes[6:], desc = 'categories', ncols=150):
    keyword_labels, results = label_keywords(cat_node, keyword_things, keyword_labels, results)

categories:   0%|                                                                                                               | 0/4 [09:04<?, ?it/s]


KeyboardInterrupt: 

## Categorical Labels Analysis
- Todo

## Optimal Game Path with MCTS
- Todo

## Save as JSONL

In [31]:
def make_list_from_dict(original_dict):
    list_of_dicts = []
    for key, sub_dict in original_dict.items():
        # Create a new dictionary that includes the key from the original dict
        new_dict = {"key": key}
        # Update the new dictionary with the key-value pairs from the sub-dictionary
        new_dict.update(sub_dict)
        # Append the new dictionary to the list
        list_of_dicts.append(new_dict)
    return list_of_dicts

labeled_keywords_list = make_list_from_dict(keyword_labels)

In [32]:
with open("20qs-data/labeled_keywords.jsonl", "w") as f:
    for i in labeled_keywords_list:
        json.dump(i, f)
        f.write('\n')

with open("20qs-data/labeling_results.jsonl", "w") as f:
    json.dump(results, f)